In [5]:
import numpy as np
import pandas as pd
import pickle
from scipy import stats
import time
import sys
sys.path.append("../")
from Schemas.stats.schema import gen_stats_light_schema
from Evaluation.training import train_one_stats, test_trained_BN_on_stats
from Join_scheme.data_prepare import process_stats_data, update_stats_data
from BayesCard.Models.Bayescard_BN import Bayescard_BN
from BayesCard.Evaluation.cardinality_estimation import parse_query_single_table

In [6]:
data_folder = "/Users/ziniuw/Desktop/research/Learned_QO/data/stats_simplified/{}.csv"

In [7]:
def read_table_csv(table_obj, csv_seperator=',', stats=True):
    """
    Reads csv from path, renames columns and drops unnecessary columns
    """
    if stats:
        df_rows = pd.read_csv(table_obj.csv_file_location)
    else:
        df_rows = pd.read_csv(table_obj.csv_file_location, header=None, escapechar='\\', encoding='utf-8',
                              quotechar='"',
                              sep=csv_seperator)
    df_rows.columns = [table_obj.table_name + '.' + attr for attr in table_obj.attributes]

    for attribute in table_obj.irrelevant_attributes:
        df_rows = df_rows.drop(table_obj.table_name + '.' + attribute, axis=1)

    return df_rows.apply(pd.to_numeric, errors="ignore")


def timestamp_transorform(time_string, start_date="2010-07-19 00:00:00"):
    start_date_int = time.strptime(start_date, "%Y-%m-%d %H:%M:%S")
    time_array = time.strptime(time_string, "%Y-%m-%d %H:%M:%S")
    return int(time.mktime(time_array)) - int(time.mktime(start_date_int))


def get_data_by_date(data_path, time_date="2014-01-01 00:00:00"):
    time_value = timestamp_transorform(time_date)
    if not data_path.endswith(".csv"):
        data_path += "/{}.csv"
    schema = gen_stats_light_schema(data_path)
    before_data = dict()
    after_data = dict()
    for table_obj in schema.tables:
        table_name = table_obj.table_name
        df_rows = read_table_csv(table_obj)
        idx = len(df_rows)
        for attribute in df_rows.columns:
            if "Date" in attribute:
                idx = np.searchsorted(df_rows[attribute].values, time_value)
                break
                
        before_data[table_name] = df_rows[:idx] if idx > 0 else None
        after_data[table_name] = df_rows[idx:] if idx < len(df_rows) else None
    return before_data, after_data

def get_data(data_path):
    if not data_path.endswith(".csv"):
        data_path += "/{}.csv"
    schema = gen_stats_light_schema(data_path)
    data = dict()
    for table_obj in schema.tables:
        table_name = table_obj.table_name
        df_rows = read_table_csv(table_obj)
        data[table_name] = df_rows
    return data


In [8]:
before_data, after_data = get_data_by_date(data_folder)
all_data = get_data(data_folder)

In [9]:
model_path = "/home/ubuntu/data_CE/CE_scheme_models/update/"
data, null_values, key_attrs, table_buckets, equivalent_keys, schema, bin_size = process_stats_data(data_folder,
                                        model_path, 200, "sub_optimal", False, actual_data=before_data)


bucketizing equivalent key group: {'votes.PostId', 'postLinks.PostId', 'posts.Id', 'comments.PostId', 'tags.ExcerptPostId', 'postLinks.RelatedPostId', 'postHistory.PostId'}
bucketizing equivalent key group: {'badges.UserId', 'users.Id', 'comments.UserId', 'postHistory.UserId', 'posts.OwnerUserId', 'votes.UserId'}


In [ ]:
t_name = "postHistory"
bn = Bayescard_BN(t_name, key_attrs[t_name], bin_size[t_name], null_values=null_values[t_name])
bn.build_from_data(data[t_name])

In [11]:
model_path = "/Users/ziniuw/Desktop/research/Learned_QO/data/saved_models/"
train_one_stats("stats", data_folder, model_path, 2, 200, "sub_optimal", True, actual_data=before_data)

bucketizing equivalent key group: {'votes.PostId', 'postLinks.PostId', 'posts.Id', 'comments.PostId', 'tags.ExcerptPostId', 'postLinks.RelatedPostId', 'postHistory.PostId'}
bucketizing equivalent key group: {'badges.UserId', 'users.Id', 'comments.UserId', 'postHistory.UserId', 'posts.OwnerUserId', 'votes.UserId'}
badges
Discretizing table takes 0.015569925308227539 secs
Structure learning took 0.21733689308166504 secs.
done, parameter learning took 0.06353521347045898 secs.
votes
Discretizing table takes 1.2292289733886719 secs
Structure learning took 1.297508955001831 secs.
done, parameter learning took 0.0673990249633789 secs.
postHistory
Discretizing table takes 0.06839895248413086 secs
Structure learning took 1.7427341938018799 secs.
done, parameter learning took 0.12562084197998047 secs.
posts
Discretizing table takes 0.2793567180633545 secs
Structure learning took 1.6007959842681885 secs.
done, parameter learning took 0.16286301612854004 secs.
users
Discretizing table takes 1.241

In [12]:
with open("/Users/ziniuw/Desktop/research/Learned_QO/data/saved_models/buckets.pkl", "rb") as f:
    buckets = pickle.load(f)
with open("/Users/ziniuw/Desktop/research/Learned_QO/data/saved_models/model_stats_sub_optimal_200.pkl", "rb") as f:
    FJmodel = pickle.load(f)

In [13]:
query_file = "/Users/ziniuw/Desktop/research/Learned_QO/data/stats_simplified/stats_CEB_sub_queries.sql"
with open(query_file, "r") as f:
    queries = f.readlines()


In [14]:
qerror = []
latency = []
pred = []
for i, query_str in enumerate(queries):
    query = query_str.split("||")[0][:-1]
    print("========================")
    true_card = int(query_str.split("||")[-1])
    t = time.time()
    res = FJmodel.get_cardinality_bound_one(query)
    pred.append(res)
    latency.append(time.time() - t)
    qerror.append(res/true_card)
    print(f"estimating query {i}: predicted {res}, true_card {true_card}, qerror {res/true_card}, latency {time.time() - t}")

estimating query 0: predicted 54920.0, true_card 79851, qerror 0.6877809920977821, latency 0.010627269744873047
estimating query 1: predicted 6634616.999999996, true_card 10220614, qerror 0.6491407463387225, latency 0.013267993927001953
estimating query 2: predicted 2224733.0, true_card 1458075, qerror 1.5258014848344563, latency 0.008116960525512695
estimating query 3: predicted 2256146.713923665, true_card 1709781, qerror 1.3195530386193701, latency 0.009044885635375977
estimating query 4: predicted 5774461.999999999, true_card 7491903, qerror 0.7707603795724529, latency 0.00740504264831543
estimating query 5: predicted 577314.3126980041, true_card 428612, qerror 1.3469392193825747, latency 0.008238077163696289
estimating query 6: predicted 33096680.247657806, true_card 55900138, qerror 0.5920679524558206, latency 0.007537126541137695
estimating query 7: predicted 7191.000000000002, true_card 10972, qerror 0.6553955523149838, latency 0.005619049072265625
estimating query 8: predicted

estimating query 58: predicted 24764.87604491237, true_card 31965, qerror 0.7747497589523656, latency 0.010013818740844727
estimating query 59: predicted 24079.479201561866, true_card 26836, qerror 0.8972827247563671, latency 0.018220901489257812
estimating query 60: predicted 2243674.2445299625, true_card 2704241, qerror 0.8296872373911802, latency 0.004176139831542969
estimating query 61: predicted 24802.214955523268, true_card 32918, qerror 0.7534544916314256, latency 0.0038542747497558594
estimating query 62: predicted 62888.13620660873, true_card 86112, qerror 0.7303063011729926, latency 0.004645824432373047
estimating query 63: predicted 2143942.3005878557, true_card 2488080, qerror 0.8616854363958778, latency 0.005914211273193359
estimating query 64: predicted 949396.0, true_card 1056687, qerror 0.8984647298585106, latency 0.0023779869079589844
estimating query 65: predicted 22294.395093399944, true_card 20334, qerror 1.0964097124717196, latency 0.003798961639404297
estimating q

estimating query 117: predicted 887150.3009236722, true_card 595820, qerror 1.488956901285073, latency 0.018607139587402344
estimating query 118: predicted 1361695.979874585, true_card 1187068, qerror 1.1471086575281155, latency 0.005686044692993164
estimating query 119: predicted 949396.0, true_card 1056687, qerror 0.8984647298585106, latency 0.002401113510131836
estimating query 120: predicted 24969.0, true_card 34789, qerror 0.7177268676880623, latency 0.002238035202026367
estimating query 121: predicted 1177408.6521548855, true_card 1486002, qerror 0.7923331544337662, latency 0.004884004592895508
estimating query 122: predicted 24748.17956538627, true_card 37048, qerror 0.668003119342104, latency 0.004334926605224609
estimating query 123: predicted 54920.0, true_card 79851, qerror 0.6877809920977821, latency 0.0010340213775634766
estimating query 124: predicted 121367108.5810282, true_card 232039659, qerror 0.5230446773800344, latency 0.006112813949584961
estimating query 125: pred

estimating query 177: predicted 706.9719693599908, true_card 913, qerror 0.7743395064183908, latency 0.009896993637084961
estimating query 178: predicted 53359.525123096995, true_card 68895, qerror 0.7745050456941287, latency 0.003949642181396484
estimating query 179: predicted 30071842.999999985, true_card 91539518, qerror 0.32851214051618655, latency 0.009258031845092773
estimating query 180: predicted 168982.08176592021, true_card 70700, qerror 2.3901284549635107, latency 0.011445999145507812
estimating query 181: predicted 4716207.434891367, true_card 1984289, qerror 2.376774469289185, latency 0.005767822265625
estimating query 182: predicted 29035.63870167997, true_card 18346, qerror 1.5826686308557707, latency 0.009877920150756836
estimating query 183: predicted 19219805.679587316, true_card 3169724, qerror 6.063558114077855, latency 0.011773347854614258
estimating query 184: predicted 211320.2737914746, true_card 345663, qerror 0.611347681966177, latency 0.008586883544921875
est

estimating query 242: predicted 22287.316910170237, true_card 5044, qerror 4.418579879097985, latency 0.009063959121704102
estimating query 243: predicted 171221.11993579773, true_card 1717, qerror 99.7210948956306, latency 0.014568090438842773
estimating query 244: predicted 214695.99999999997, true_card 213642, qerror 1.0049334868611977, latency 0.004837989807128906
estimating query 245: predicted 1039890.0, true_card 1331347, qerror 0.7810811155919531, latency 0.0025718212127685547
estimating query 246: predicted 137437.71052934736, true_card 150600, qerror 0.9126009995308589, latency 0.0029981136322021484
estimating query 247: predicted 27412.077844539683, true_card 41569, qerror 0.6594355852808507, latency 0.0035991668701171875
estimating query 248: predicted 1039890.0, true_card 915414, qerror 1.1359778198716646, latency 0.004544973373413086
estimating query 249: predicted 213711.70844690394, true_card 209583, qerror 1.0196996342589997, latency 0.007542133331298828
estimating que

...

estimating query 2276: predicted 3175126.5913756485, true_card 211971, qerror 14.979061245998974, latency 0.003802776336669922
estimating query 2277: predicted 2721842.1734249536, true_card 1122938, qerror 2.423857927530241, latency 0.0020449161529541016
estimating query 2278: predicted 10855133.865634272, true_card 3452526, qerror 3.1441135752878537, latency 0.00940704345703125
estimating query 2279: predicted 158779.56753747456, true_card 29591, qerror 5.3658060740588205, latency 0.005717754364013672
estimating query 2280: predicted 12994152.0, true_card 4255848, qerror 3.05324626255449, latency 0.0034050941467285156
estimating query 2281: predicted 10872868.0, true_card 70129559, qerror 0.15503973153460154, latency 0.009186983108520508
estimating query 2282: predicted 2398000.6588536594, true_card 691208, qerror 3.469289503092643, latency 0.006033182144165039
estimating query 2283: predicted 10846177.542969456, true_card 81958485, qerror 0.1323374577137371, latency 0.007737874984741

estimating query 2348: predicted 500531.23190874996, true_card 253217, qerror 1.9766888949349766, latency 0.02544093132019043
estimating query 2349: predicted 6560647.291489274, true_card 5543342, qerror 1.1835184066740378, latency 0.009793996810913086
estimating query 2350: predicted 92739.31349551963, true_card 63198, qerror 1.4674406388733763, latency 0.009550094604492188
estimating query 2351: predicted 3294989.6201880327, true_card 136431, qerror 24.151326459441275, latency 0.007097959518432617
estimating query 2352: predicted 2687213.2299221866, true_card 520276, qerror 5.1649763393317905, latency 0.024857044219970703
estimating query 2353: predicted 10845791.943054449, true_card 1827833, qerror 5.93368865922349, latency 0.010057926177978516
estimating query 2354: predicted 161388.56748889745, true_card 19469, qerror 8.289514997632002, latency 0.006885051727294922
estimating query 2355: predicted 9699342.551246593, true_card 2763952, qerror 3.5092297374363204, latency 0.026454925

estimating query 2416: predicted 76467.24852392465, true_card 55484, qerror 1.3781855764531152, latency 0.00783991813659668
estimating query 2417: predicted 125857.18003851757, true_card 65334, qerror 1.9263657519594326, latency 0.003526926040649414
estimating query 2418: predicted 171841.45284697707, true_card 164951, qerror 1.0417727255183484, latency 0.007646083831787109
estimating query 2419: predicted 1039428.3713686675, true_card 987389, qerror 1.0527040217874288, latency 0.008306264877319336
estimating query 2420: predicted 54003.0, true_card 75910, qerror 0.7114082466078251, latency 0.002641916275024414
estimating query 2421: predicted 163285.14832577817, true_card 16583, qerror 9.846538522931807, latency 0.004189968109130859
estimating query 2422: predicted 2426569.6371417586, true_card 185793, qerror 13.060608511309676, latency 0.0049059391021728516
estimating query 2423: predicted 651548.45048051, true_card 146285, qerror 4.453966233588611, latency 0.00887298583984375
estima

estimating query 2474: predicted 2736817.5012277425, true_card 67369732, qerror 0.04062384426922973, latency 0.013869047164916992
estimating query 2475: predicted 26818513.342318006, true_card 289290682, qerror 0.09270438009585807, latency 0.015101909637451172
estimating query 2476: predicted 46481443.154860005, true_card 155696936, qerror 0.29853794396352157, latency 0.016985177993774414
estimating query 2477: predicted 39926166.8889359, true_card 1297422, qerror 30.773462211166375, latency 0.014890909194946289
estimating query 2478: predicted 3179897.961960524, true_card 12492036, qerror 0.25455401841305325, latency 0.011044740676879883
estimating query 2479: predicted 2736817.5012277425, true_card 21582838, qerror 0.12680526542560078, latency 0.015145063400268555
estimating query 2480: predicted 24270672.40716143, true_card 96498890, qerror 0.25151245166821534, latency 0.01573491096496582
estimating query 2481: predicted 46481443.154860005, true_card 524182774, qerror 0.088674114183

estimating query 2540: predicted 239652.87947577654, true_card 175223, qerror 1.367702182223661, latency 0.010298967361450195
estimating query 2541: predicted 49734.142963431324, true_card 38044, qerror 1.3072795437764515, latency 0.01175379753112793
estimating query 2542: predicted 115787.27484966909, true_card 146996, qerror 0.7876899701329906, latency 0.014509201049804688
estimating query 2543: predicted 708231.2170188845, true_card 90846, qerror 7.795953779130446, latency 0.009762763977050781
estimating query 2544: predicted 3819083.198135077, true_card 51304, qerror 74.4402619315273, latency 0.006876707077026367
estimating query 2545: predicted 1024651.7817532377, true_card 251465, qerror 4.074729213819966, latency 0.009722232818603516
estimating query 2546: predicted 195948.3839197432, true_card 33366, qerror 5.872696275242559, latency 0.01128697395324707
estimating query 2547: predicted 6200272.427053219, true_card 668362, qerror 9.276817693186056, latency 0.010223865509033203
e

In [ ]:
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(qerror, i)}")
print(f"total inference time: {np.sum(latency)}")

In [15]:

model_path = "/Users/ziniuw/Desktop/research/Learned_QO/data/saved_models/"
table_buckets = FJmodel.table_buckets
null_values = FJmodel.null_value
data, table_buckets, null_values = update_stats_data(data_folder, model_path, buckets, table_buckets,
                                                     null_values, False, after_data)

tags does not have data to update
updating equivalent key group: {'votes.PostId', 'postLinks.PostId', 'posts.Id', 'comments.PostId', 'tags.ExcerptPostId', 'postLinks.RelatedPostId', 'postHistory.PostId'}
tags.ExcerptPostId
updating equivalent key group: {'badges.UserId', 'users.Id', 'comments.UserId', 'postHistory.UserId', 'posts.OwnerUserId', 'votes.UserId'}


In [ ]:
def test_trained_BN_on_stats(bn, t_name):
    queries = {
        "posts": "SELECT COUNT(*) FROM posts as p WHERE posts.CommentCount<=18 AND posts.CreationDate>='2010-07-23 07:27:31'::timestamp AND posts.CreationDate<='2014-09-09 01:43:00'::timestamp",
        "comments": "SELECT COUNT(*) FROM comments as c WHERE comments.CreationDate>='2010-08-05 00:36:02'::timestamp AND comments.CreationDate<='2014-09-08 16:50:49'::timestamp",
        "postHistory": "SELECT COUNT(*) FROM postHistory as ph WHERE postHistory.PostHistoryTypeId=1 AND postHistory.CreationDate>='2010-09-14 11:59:07'::timestamp",
        "votes": "SELECT COUNT(*) FROM votes as v WHERE votes.VoteTypeId=2 AND votes.CreationDate<='2014-09-10 00:00:00'::timestamp",
        "postLinks": "SELECT COUNT(*) FROM postLinks as pl WHERE postLinks.LinkTypeId=1 AND postLinks.CreationDate>='2011-09-03 21:00:10'::timestamp AND postLinks.CreationDate<='2014-07-30 21:29:52'::timestamp",
        "users": "SELECT COUNT(*) FROM users as u WHERE users.DownVotes>=0 AND users.DownVotes<=0 AND users.UpVotes>=0 AND users.UpVotes<=31 AND users.CreationDate<='2014-08-06 20:38:52'::timestamp",
        "badges": "SELECT COUNT(*) FROM badges as b WHERE badges.Date>='2010-09-26 12:17:14'::timestamp",
        "tags": "SELECT COUNT(*) FROM tags"
    }

    true_cards = {
        "posts": 90764,
        "comments": 172156,
        "postHistory": 42308,
        "votes": 261476,
        "postLinks": 8776,
        "users": 37062,
        "badges": 77704,
        "tags": 1032
    }

    bn.init_inference_method()
    bn.infer_algo = "exact-jit"
    query = parse_query_single_table(queries[t_name], bn)
    pred = bn.query(query)
    print(pred)
    assert min(pred, true_cards[t_name]) / max(pred, true_cards[t_name]) <= 1.5, f"Qerror too large, we have predition" \
                                                                        f"{pred} for true card {true_cards[t_name]}"

    query = parse_query_single_table(queries[t_name], bn)
    _, id_probs = bn.query_id_prob(query, bn.id_attributes)
    print(np.sum(id_probs))
    if t_name not in ['votes', 'tags']:
        assert min(pred, np.sum(id_probs)) / max(pred, np.sum(id_probs)) <= 1.5, "query_id_prob is incorrect"

In [ ]:
[1,2,3].index(4)

In [ ]:
t_name = "postHistory"
bn = FJmodel.bns[t_name]
bn.null_values = null_values[t_name]
print(len(data[t_name]))
bn.update_from_data(data[t_name])
test_trained_BN_on_stats(bn, t_name)

In [16]:
for table in FJmodel.schema.tables:
    t_name = table.table_name
    print(t_name)
    if t_name in data and data[t_name] is not None:
        bn = FJmodel.bns[t_name]
        bn.null_values = null_values[t_name]
        bn.update_from_data(data[t_name])
        #test_trained_BN_on_stats(bn, t_name)

badges
Discretizing table took 0.024923086166381836 secs.
old value indexing
(-1, 1) 120
-1 105
new_cpd.state_names:  1
new_index: 1
new_cpd.state_names:  105
new_index: 105
new_cpd.value:  (1, 105)
(-1, 1) 1
-1 105
old value indexing
-1 105
new_cpd.state_names:  105
new_index: 105
new_cpd.value:  (105,)
-1 105
done, incremental parameter updating took 0.09226179122924805 secs.
votes


Discretizing table took 0.23058009147644043 secs.
old value indexing
(-1, 1) 12
-1 12
new_cpd.state_names:  10
new_index: 10
new_cpd.state_names:  10
new_index: 10
new_cpd.value:  (10, 10)
(-1, 1) 10
-1 10
old value indexing
(-1, 1) 89
-1 46
new_cpd.state_names:  30
new_index: 30
new_cpd.state_names:  46
new_index: 46
new_cpd.value:  (30, 46)
(-1, 1) 30
-1 46
old value indexing
-1 46
new_cpd.state_names:  46
new_index: 46
new_cpd.value:  (46,)
-1 46
old value indexing
(-1, 1) 105
-1 89
new_cpd.state_names:  99
new_index: 99
new_cpd.state_names:  30
new_index: 30
new_cpd.value:  (99, 30)
(-1, 1) 99
-1 30
old value indexing
(-1, 1) 12
-1 105
new_cpd.state_names:  10
new_index: 10
new_cpd.state_names:  99
new_index: 99
new_cpd.value:  (10, 99)
(-1, 1) 10
-1 99
done, incremental parameter updating took 0.07032203674316406 secs.
postHistory
Discretizing table took 0.03910708427429199 secs.


old value indexing
(-1, 1) 120
-1 106
new_cpd.state_names:  4
new_index: 4
new_cpd.state_names:  103
new_index: 103
new_cpd.value:  (4, 103)
(-1, 1) 4
-1 103
old value indexing
-1 25
new_cpd.state_names:  25
new_index: 25
new_cpd.value:  (25,)
-1 25
old value indexing
(-1, 1) 46
-1 120
new_cpd.state_names:  45
new_index: 45
new_cpd.state_names:  4
new_index: 4
new_cpd.value:  (45, 4)
(-1, 1) 45
-1 4
old value indexing
(-1, 1) 106
-1 25
new_cpd.state_names:  103
new_index: 103
new_cpd.state_names:  25
new_index: 25
new_cpd.value:  (103, 25)
(-1, 1) 103
-1 25
done, incremental parameter updating took 0.17610597610473633 secs.
posts
Discretizing table took 0.12494802474975586 secs.


old value indexing
(-1, 1) 32
-1 53
new_cpd.state_names:  13
new_index: 13
new_cpd.state_names:  53
new_index: 53
new_cpd.value:  (13, 53)
(-1, 1) 13
-1 53
old value indexing
(-1, 1) 37
-1 46
new_cpd.state_names:  34
new_index: 34
new_cpd.state_names:  42
new_index: 42
new_cpd.value:  (34, 42)
(-1, 1) 34
-1 42
old value indexing
(-1, 1) 120
-1 106
new_cpd.state_names:  4
new_index: 4
new_cpd.state_names:  103
new_index: 103
new_cpd.value:  (4, 103)
(-1, 1) 4
-1 103
old value indexing
(-1, 1) 60
-1 46
new_cpd.state_names:  21
new_index: 21
new_cpd.state_names:  42
new_index: 42
new_cpd.value:  (21, 42)
(-1, 1) 21
-1 42
old value indexing
-1 46
new_cpd.state_names:  42
new_index: 42
new_cpd.value:  (42,)
-1 42
old value indexing
(-1, 1) 106
-1 53
new_cpd.state_names:  103
new_index: 103
new_cpd.state_names:  53
new_index: 53
new_cpd.value:  (103, 53)
(-1, 1) 103
-1 53
old value indexing
(-1, 1) 7
-1 32
new_cpd.state_names:  4
new_index: 4
new_cpd.state_names:  13
new_index: 13
new_cpd.va

Discretizing table took 0.32356691360473633 secs.
old value indexing
(-1, 1) 120
-1 84
new_cpd.state_names:  6
new_index: 6
new_cpd.state_names:  96
new_index: 96
new_cpd.value:  (6, 96)
(-1, 1) 6
-1 96
old value indexing
(-1, 1) 54
-1 81
new_cpd.state_names:  11
new_index: 11
new_cpd.state_names:  57
new_index: 57
new_cpd.value:  (11, 57)
(-1, 1) 11
-1 57
old value indexing
-1 106
new_cpd.state_names:  17
new_index: 17
new_cpd.value:  (17,)
-1 17
old value indexing
(-1, 1) 84
-1 106
new_cpd.state_names:  96
new_index: 96
new_cpd.state_names:  17
new_index: 17
new_cpd.value:  (96, 17)
(-1, 1) 96
-1 17
old value indexing
(-1, 1) 81
-1 84
new_cpd.state_names:  57
new_index: 57
new_cpd.state_names:  96
new_index: 96
new_cpd.value:  (57, 96)
(-1, 1) 57
-1 96
old value indexing
(-1, 1) 80
-1 84
new_cpd.state_names:  61
new_index: 61
new_cpd.state_names:  96
new_index: 96
new_cpd.value:  (61, 96)
(-1, 1) 61
-1 96
done, incremental parameter updating took 0.09523582458496094 secs.
comments
Di

old value indexing
(-1, 1) 120
-1 46
new_cpd.state_names:  39
new_index: 39
new_cpd.state_names:  46
new_index: 46
new_cpd.value:  (39, 46)
(-1, 1) 39
-1 46
old value indexing
-1 46
new_cpd.state_names:  46
new_index: 46
new_cpd.value:  (46,)
-1 46
old value indexing
(-1, 1) 33
-1 106
new_cpd.state_names:  23
new_index: 23
new_cpd.state_names:  104
new_index: 104
new_cpd.value:  (23, 104)
(-1, 1) 23
-1 104
old value indexing
(-1, 1) 106
-1 120
new_cpd.state_names:  104
new_index: 104
new_cpd.state_names:  39
new_index: 39
new_cpd.value:  (104, 39)
(-1, 1) 104
-1 39
done, incremental parameter updating took 0.171644926071167 secs.
postLinks
Discretizing table took 0.009223222732543945 secs.
old value indexing
-1 117
new_cpd.state_names:  1
new_index: 1
new_cpd.value:  (1,)
-1 1
old value indexing
(-1, 1) 2
-1 117
new_cpd.state_names:  2
new_index: 2
new_cpd.state_names:  1
new_index: 1
new_cpd.value:  (2, 1)
(-1, 1) 2
-1 1
old value indexing
(-1, 1) 46
-1 117
new_cpd.state_names:  42
ne

In [ ]:
a = [1,2,2,3,]
a.remove(1)
a

In [ ]:
np.ones(1).astype(int)

In [ ]:
a = np.arange(10)
a = a.delete(0)
a

In [ ]:
import numpy as np
import pickle
import time
import os
import sys
sys.path.append("/home/ubuntu/CE_scheme")
from Schemas.stats.schema import gen_stats_light_schema
from Evaluation.training import train_one_stats, test_trained_BN_on_stats
from Join_scheme.data_prepare import read_table_csv, update_stats_data
from BayesCard.Models.Bayescard_BN import Bayescard_BN


def timestamp_transorform(time_string, start_date="2010-07-19 00:00:00"):
    start_date_int = time.strptime(start_date, "%Y-%m-%d %H:%M:%S")
    time_array = time.strptime(time_string, "%Y-%m-%d %H:%M:%S")
    return int(time.mktime(time_array)) - int(time.mktime(start_date_int))


def get_data_by_date(data_path, time_date="2014-01-01 00:00:00"):
    time_value = timestamp_transorform(time_date)
    if not data_path.endswith(".csv"):
        data_path += "/{}.csv"
    schema = gen_stats_light_schema(data_path)
    before_data = dict()
    after_data = dict()
    for table_obj in schema.tables:
        table_name = table_obj.table_name
        df_rows = read_table_csv(table_obj)
        idx = len(df_rows)
        for attribute in df_rows.columns:
            if "Date" in attribute:
                idx = np.searchsorted(df_rows[attribute].values, time_value)
                break

        before_data[table_name] = df_rows[:idx] if idx > 0 else None
        after_data[table_name] = df_rows[idx:] if idx < len(df_rows) else None
    return before_data, after_data


def update_one_stats(FJmodel, buckets, table_buckets, data_path, save_model_folder, save_bucket_bins=False,
                     update_BN=True, retrain_BN=False, old_data=None, validate=False):
    """
    Incrementally update the FactorJoin model
    """
    data, table_buckets, null_values = update_stats_data(data_path, save_model_folder, buckets, table_buckets,
                                                         save_bucket_bins)
    FJmodel.table_buckets = table_buckets
    if update_BN:
        # updating the single table estimator
        if retrain_BN:
            # retrain the BN based on the new and old data
            for table in FJmodel.schema.tables:
                t_name = table.table_name
                if t_name in data and data[t_name] is not None:
                    bn = Bayescard_BN(t_name, table_buckets[t_name].id_attributes, table_buckets[t_name].bin_sizes,
                                      null_values=null_values[t_name])
                    new_data = old_data[t_name].append(data[t_name], ignore_index=True)
                    bn.build_from_data(new_data)
                    if validate:
                        test_trained_BN_on_stats(bn, t_name)
                    FJmodel.bns[t_name] = bn
        else:
            # incrementally update BN
            for table in FJmodel.schema.tables:
                t_name = table.table_name
                if t_name in data and data[t_name] is not None:
                    bn = FJmodel.bns[t_name]
                    bn.null_values = null_values[t_name]
                    bn.update_from_data(data)

    model_path = save_model_folder + f"update_model.pkl"
    pickle.dump(FJmodel, open(model_path, 'wb'), pickle.HIGHEST_PROTOCOL)
    print(f"models save at {model_path}")


def eval_update(data_folder, model_path, n_dim_dist, bin_size, bucket_method, split_date="2014-01-01 00:00:00"):
    before_data, after_data = get_data_by_date(data_folder, split_date)
    print("************************************************************")
    print(f"Training the model with data before {split_date}")
    start_time = time.time()
    train_one_stats("stats", data_folder, model_path, n_dim_dist, bin_size, bucket_method, True, actual_data=before_data)
    print(f"training completed, took {time.time() - start_time} sec")

    #loading the trained model and buckets
    with open(model_path + "buckets.pkl", "rb") as f:
        buckets = pickle.load(f)
    with open(model_path + f"model_stats_{bucket_method}_{bin_size}.pkl", "rb") as f:
        FJmodel = pickle.load(f)
    print("************************************************************")
    print(f"Updating the model with data after {split_date}")
    start_time = time.time()
    table_buckets = FJmodel.table_buckets
    null_values = FJmodel.null_value
    data, table_buckets, null_values = update_stats_data(data_folder, model_path, buckets, table_buckets,
                                                         null_values, False, after_data)
    for table in FJmodel.schema.tables:
        t_name = table.table_name
        if t_name in data and data[t_name] is not None:
            bn = FJmodel.bns[t_name]
            bn.null_values = null_values[t_name]
            bn.update_from_data(data[t_name])
            #test_trained_BN_on_stats(bn, t_name)
    print(f"updating completed, took {time.time() - start_time} sec")
    model_path = model_path + f"updated_model_stats_{bucket_method}_{bin_size}.pkl"
    pickle.dump(FJmodel, open(model_path, 'wb'), pickle.HIGHEST_PROTOCOL)
    print(f"updated models save at {model_path}")

In [ ]:
data_path = "/Users/ziniuw/Desktop/research/Learned_QO/data/stats_simplified/{}.csv"
model_path = "/Users/ziniuw/Desktop/research/Learned_QO/data/saved_models"
eval_update(data_path, model_path, 2, 200, "sub_optimal")